# Sympy decomposotition code

In [2]:
import sympy as sp

In [3]:
# Define symbolic Pauli matrices
I = sp.Matrix([[1, 0], [0, 1]])
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])
pauli_matrices = [I, X, Y, Z]
pauli_labels = ["I", "X", "Y", "Z"]

In [4]:
# Function to decompose a 2x2 matrix into Pauli matrices
def decompose_2x2_symbolic(M):
    coefficients = {}
    for i, P in enumerate(pauli_matrices):
        coefficient = (1 / 2) * sp.trace(M * P)
        coefficients[pauli_labels[i]] = sp.simplify(coefficient)
    # Remove zero coefficients
    coefficients = {key: value for key, value in coefficients.items() if value != 0}
    return coefficients

In [5]:
# Function to compute the commutator
def commutator(A, B):
    return A * B - B * A

# Function to compute the anticommutator
def anticommutator(A, B):
    return A * B + B * A

def lindblad_term(O, L):
    """Dissipative Lindblad term."""
    term1 = sp.conjugate(sp.transpose(L)) * O * L # L_k^\dagger * O * L
    term2 = anticommutator(O, sp.conjugate(sp.transpose(L)) * L) / 2  # {O, L_k^\dagger L_k} / 2
    return term1 - term2

In [6]:
# Main EDP function
def EDP(H, O, Ls, gammas):
    """
    Compute the time derivative of <O> and decompose it into Pauli tensor products.

    H: Hamiltonian (symbolic matrix)
    O: Observable (symbolic matrix)
    Ls: List of Lindblad operators (symbolic matrices)
    gammas: List of decay rates associated with Lindblad operators (symbolic values)

    Returns:
        Dictionary of coefficients for the Pauli tensor product decomposition.
    """
    # Unitary part: -i * [H, O]
    unitary_part = (-1) * 1j * commutator(O, H)

    # Dissipative part
    dissipative_part = sp.zeros(2)
    for gamma_k, L_k in zip(gammas, Ls):
        L = gamma_k * lindblad_term(O, L_k)
        dissipative_part += L

    # Total time derivative
    dO_dt = unitary_part + dissipative_part

    # Decompose into Pauli tensor products
    #decomposition = decompose_4x4_symbolic_new(dO_dt)
    decomposition = decompose_2x2_symbolic(dO_dt)
    return decomposition

# 1 qubit

In [7]:
# Define the Lindblad dissipation
gamma1, gamma2, gamma3 = sp.symbols("gamma1 gamma2 gamma3", real=True, positive=True)
gammas = [gamma1, gamma2, gamma3]

L1 = (X - 1j*Y)/2
L2 = Z
L3 = (X + 1j*Y)/2
Ls = [L1, L2, L3]

In [8]:
# Define the Hamiltonian with external magnetic field
JX,JY,JZ = sp.symbols('JX,JY,JZ')

H = JX* X + JY* Y + JZ*Z
for i, P1 in enumerate(pauli_matrices):
    #for j, P2 in enumerate(pauli_matrices):
        O = P1
        Eq = EDP(H, O, Ls, gammas)
        print(f'{pauli_labels[i]} {Eq}')

I {}
X {'X': -0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3, 'Y': -2.0*JZ, 'Z': 2.0*JY}
Y {'X': 2.0*JZ, 'Y': -0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3, 'Z': -2.0*JX}
Z {'I': -1.0*gamma1 + 1.0*gamma3, 'X': -2.0*JY, 'Y': 2.0*JX, 'Z': -1.0*gamma1 - 1.0*gamma3}


In [34]:
# Define the Hamiltonian with external magnetic field
JX,JY,JZ = sp.symbols('JX,JY,JZ')

H = JX* X + JY* Y + JZ*Z
countador = 0
for i, P1 in enumerate(pauli_matrices):
    O = P1
    Eq = EDP(H, O, Ls, gammas)

    saida =''
    if len(Eq) !=0 :
        saida  = f'LOSS_edo += (dX_dt[:,{countador}:{countador+1}] - (' 
        for key, value in Eq.items():
            saida += f' + ({value})*{key}'
        saida += '))**2'
        print(saida)
        countador +=1

LOSS_edo += (dX_dt[:,0:1] - ( + (-0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3)*X + (-2.0*JZ)*Y + (2.0*JY)*Z))**2
LOSS_edo += (dX_dt[:,1:2] - ( + (2.0*JZ)*X + (-0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3)*Y + (-2.0*JX)*Z))**2
LOSS_edo += (dX_dt[:,2:3] - ( + (-1.0*gamma1 + 1.0*gamma3)*I + (-2.0*JY)*X + (2.0*JX)*Y + (-1.0*gamma1 - 1.0*gamma3)*Z))**2
